In [1]:
import pandas as pd
import requests
from plyer import notification
from datetime import datetime
import sqlite3
from tqdm import tqdm

In [2]:
def alerta(nivel, base, etapa):
    if nivel == 1:
        titulo = "Alerta Baixo"
    elif nivel == 2:
        titulo = "Alerta Médio"
    elif nivel == 3:
        titulo = "Alerta Alto"
    else:
        titulo = "Alerta Desconhecido"

    data_atual = datetime.now().strftime("%d/%m/%Y %H:%M:%S")

    mensagem = f"Falha no carregamento da base {base} na etapa {etapa}\nData: {data_atual}"

    notification.notify(
        title=titulo,
        message=mensagem,
        timeout=20
    )

In [3]:
def salva_bd(df, nome_tabela):
    conn = sqlite3.connect('data_pipeline.db')
    df.to_sql(nome_tabela, conn, if_exists='replace', index=False)
    conn.close()

def carrega_bd(nome_tabela):
    conn = sqlite3.connect('data_pipeline.db')
    query = f"SELECT * FROM {nome_tabela}"
    df = pd.read_sql(query, conn)
    conn.close()
    return df

In [4]:
def get_json_api(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        alerta(3, url, "Requisição", response.text)
        raise Exception(f"Erro na requisição: {response.status_code}")

def get_bancos():
    url = "https://brasilapi.com.br/api/banks/v1"
    data_json = get_json_api(url)
    df_bancos = pd.DataFrame(data_json)
    return df_bancos

def get_pix_participants():
    url = "https://brasilapi.com.br/api/pix/v1/participants"
    data_json = get_json_api(url)
    df_pix = pd.DataFrame(data_json)
    return df_pix

def get_taxas():
    url = "https://brasilapi.com.br/api/taxas/v1"
    data_json = get_json_api(url)
    df_taxas = pd.DataFrame(data_json)
    return df_taxas

In [5]:
def etapa_extracao():
    try:
        df_bancos = get_bancos()
        salva_bd(df_bancos, "bancos")
        
    except Exception as e:
        alerta(3, "bancos", "EXTRACAO", str(e))

    try:
        df_pix = get_pix_participants()
        salva_bd(df_pix, "pix_participants")
    except Exception as e:
        alerta(3, "pix_participants", "EXTRACAO", str(e))

    try:
        df_taxas = get_taxas()
        salva_bd(df_taxas, "taxas")
    except Exception as e:
        alerta(3, "taxas", "EXTRACAO", str(e))

In [6]:
def etapa_transformacao():
    try:
        df_bancos = carrega_bd("bancos")
        # Aqui você pode adicionar qualquer transformação necessária
        salva_bd(df_bancos, "bancos_transformados")
    except Exception as e:
        alerta(2, "bancos_transformados", "TRANSFORMACAO", str(e))

    try:
        df_pix = carrega_bd("pix_participants")
        # Transformações específicas para df_pix
        salva_bd(df_pix, "pix_participants_transformados")
    except Exception as e:
        alerta(2, "pix_participants_transformados", "TRANSFORMACAO", str(e))

    try:
        df_taxas = carrega_bd("taxas")
        # Transformações específicas para df_taxas
        salva_bd(df_taxas, "taxas_transformadas")
    except Exception as e:
        alerta(2, "taxas_transformadas", "TRANSFORMACAO", str(e))

In [11]:
etapa_extracao()

In [12]:
etapa_transformacao()

In [20]:
df = carrega_bd('pix_participants_transformados')
df.head()

,ispb,nome,nome_reduzido,modalidade_participacao,tipo_participacao,inicio_operacao
0,00000000,BANCO DO BRASIL S.A.,BCO DO BRASIL S.A.,PDCT,DRCT,2020-11-03T09:30:00.000Z
1,00000208,BRB - BANCO DE BRASILIA S.A.,BRB - BCO DE BRASILIA S.A.,PDCT,DRCT,2020-11-03T09:30:00.000Z
2,00068987,COOPERATIVA DE CRÉDITO ARACREDI LTDA. - SICOOB...,CC ARACREDI LTDA.,PDCT,IDRT,2020-11-03T09:30:00.000Z
3,00075847,COOPERATIVA DE CRÉDITO UNICRED CENTRO-SUL LTDA...,CC UNICRED CENTRO-SUL LTDA - UNICRED CENTRO-SUL,PDCT,IDRT,2020-11-03T09:30:00.000Z
4,00106180,CENTRAL DAS COOPERATIVAS DE CRÉDITO DOS ESTADO...,"CCC DOS EST DE MT, MS E MUN DE CACOAL/RO",PDCT,IDRT,2020-11-03T09:30:00.000Z


In [21]:
df.shape

(834, 6)